In [2]:
from backend import ercot_df

ercot_df.head()

100%|██████████| 6/6 [00:02<00:00,  2.38it/s]


,Oper Day,Interval Ending,HB_BUSAVG,HB_HOUSTON,HB_HUBAVG,HB_NORTH,HB_PAN,HB_SOUTH,HB_WEST,LZ_AEN,LZ_CPS,LZ_HOUSTON,LZ_LCRA,LZ_NORTH,LZ_RAYBN,LZ_SOUTH,LZ_WEST
0,11/01/2024,0015,20.17,20.48,21.74,17.70,-24.08,22.52,26.25,21.97,23.14,20.56,22.46,17.61,17.33,22.46,40.71
1,11/01/2024,0030,21.44,21.67,22.65,19.53,-23.76,23.24,26.14,22.82,23.72,21.73,23.22,19.49,19.25,23.18,37.37
2,11/01/2024,0045,19.52,19.88,21.38,16.61,-24.31,22.29,26.73,21.64,23.03,19.98,22.27,16.53,16.17,22.20,43.98
3,11/01/2024,0100,19.79,20.22,22.07,16.19,-24.53,23.20,28.68,22.40,24.10,20.35,23.17,16.10,15.65,23.08,49.90
4,11/01/2024,0115,19.98,20.45,22.41,16.16,-24.60,23.61,29.44,22.76,24.58,20.58,23.57,16.06,15.59,23.49,52.13


In [3]:
from tovowpal import df_to_vw

vw = df_to_vw(ercot_df, label='HB_BUSAVG')

<class 'list'>


c:\Users\alexj\polybox\Master\Semester 1\Boiler Quant Finance Group\Project\bqfg-online-learning\venv\lib\site-packages\vowpalwabbit\dftovw.py:51: UserWarning: Name 'Oper Day' was not a valid feature/namespace name. It has been renamed 'Oper_Day'
  warnings.warn(
c:\Users\alexj\polybox\Master\Semester 1\Boiler Quant Finance Group\Project\bqfg-online-learning\venv\lib\site-packages\vowpalwabbit\dftovw.py:51: UserWarning: Name 'Interval Ending' was not a valid feature/namespace name. It has been renamed 'Interval_Ending'
  warnings.warn(


TypeError: In argument 'label' of 'SimpleLabel', column 'HB_BUSAVG' should be either of the following type(s): 'int', 'float'.